In [ ]:
!pip install itranslate
!pip install tqdm

In [ ]:
import pandas as pd
import glob
import os
import re
import emoji
import functools
import operator

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
ROOT_FOLDER = './'

In [ ]:
full_df = pd.read_csv(os.path.join(ROOT_FOLDER, 'data', 'full_df.csv'))
full_df

,author_id,following_count,like_count,listed_count,quote_count,retweet_count,text,tweet_count,tweet_created_at,tweet_id,user_created_at,user_verified,username,country,period
0,3.605107e+08,1012.0,3.0,106.0,0.0,1.0,Iran seeks tech in Sweden for nuclear weapons ...,213639.0,2021-04-29T19:34:55.000Z,1.387850e+18,2011-08-23T09:53:35.000Z,0.0,aj_jobs,Bahrain,before
1,3.605107e+08,1012.0,0.0,106.0,0.0,1.0,"EU, UN call on Abbas to set new date for Pales...",213639.0,2021-04-30T18:16:01.000Z,1.388200e+18,2011-08-23T09:53:35.000Z,0.0,aj_jobs,Bahrain,before
2,1.145710e+18,533.0,5.0,1.0,0.0,0.0,Who has destroyed this region?\n\nWho is prop...,15125.0,2021-04-30T16:46:27.000Z,1.388170e+18,2019-07-01T15:17:35.000Z,0.0,AhamadNooh,Bahrain,before
3,3.266501e+08,1218.0,0.0,1.0,0.0,0.0,@PMOIndia @Swamy39 @TheAjitDovalNSA India govt...,3578.0,2021-05-02T13:50:47.000Z,1.388850e+18,2011-06-30T07:56:06.000Z,0.0,jain_sushil,Bahrain,before
4,3.605107e+08,1012.0,0.0,106.0,0.0,0.0,Fmr. Iran nuke chief: Natanz explosion was 5th...,213639.0,2021-05-02T09:50:12.000Z,1.388790e+18,2011-08-23T09:53:35.000Z,0.0,aj_jobs,Bahrain,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16414,1.264182e+18,149.0,0.0,1.0,0.0,0.0,#Palestine #Apartheid #IsraeliCrimes #Palestin...,2361.0,2021-06-04T23:42:30.000Z,1.400961e+18,2020-05-23T13:10:18.000Z,0.0,AndPakistani,United Arab Emirates,after
16415,2.819481e+08,353.0,47.0,836.0,2.0,7.0,The Abraham Accord is not a Netanyahu accord a...,69013.0,2021-06-04T16:05:33.000Z,1.400846e+18,2011-04-14T08:51:07.000Z,0.0,Abdulkhaleq_UAE,United Arab Emirates,after
16416,9.644186e+17,1084.0,0.0,1.0,0.0,0.0,@insidegunner Jealousy!! Kills! Free Palestine...,2182.0,2021-06-04T15:59:13.000Z,1.400845e+18,2018-02-16T08:38:08.000Z,0.0,Ozilnation,United Arab Emirates,after
16417,1.189494e+18,1261.0,1.0,12.0,0.0,0.0,@david_goold And I believe the Jews fled the R...,58883.0,2021-06-04T14:32:07.000Z,1.400823e+18,2019-10-30T10:48:55.000Z,0.0,PinkTaxiRunner,United Arab Emirates,after


In [ ]:
def remove_emojis(tweet):
  em_split_emoji = emoji.get_emoji_regexp().split(tweet)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  tweet = functools.reduce(operator.concat, em_split_whitespace)
  tweet = ' '.join(c for c in tweet if c not in emoji.UNICODE_EMOJI['en'])
  return tweet

def pre_translate_cleaner(tweet):
  # print(tweet)
  tweet = re.sub("@[A-Za-z0-9]+", "", tweet) #Remove @ sign
  tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
  tweet = " ".join(tweet.split())
  tweet = remove_emojis(tweet)
  tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
  return tweet

In [ ]:
full_df['clean_text'] = full_df['text'].apply(lambda x: pre_translate_cleaner(x))

In [ ]:
from itranslate import itranslate as itrans

def translate(text):
  time.sleep(1)
  return itrans(text, to_lang='en')

In [ ]:
from tqdm import tqdm
import time

df = pd.DataFrame()
for i in tqdm(range(full_df.shape[0])):
  row = full_df.iloc[i]
  clean_text = row['clean_text']
  trans_text = translate(clean_text)
  df = df.append(row.append(pd.Series(trans_text, index=['trans_text'])), ignore_index=True)
  if df.shape[0] % 10 == 0:
    df.to_csv(ROOT_FOLDER + '/data/full_w_trans.csv', mode='a', header=True if i==9 else False)
    df = pd.DataFrame()